In [2]:
!pip install Jaydebeapi
!pip install PyGithub

You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


## Fazer o download do Driver JDBC

In [1]:
from github import Github
import requests
import os

base_path = "/mnt/notebooks/Individual_Desafio1_FIAP"

g = Github()
asset = g.get_repo('timveil/hive-jdbc-uber-jar').get_latest_release().get_assets()[0]
url = asset.browser_download_url
print(asset.name)
print(url)

response = requests.get(url)
open(f'{base_path}/driver/{asset.name}', 'wb').write(response.content)
SSresponse.close()

hive-jdbc-uber-2.6.3.0-235.jar
https://github.com/timveil/hive-jdbc-uber-jar/releases/download/v1.8-2.6.3/hive-jdbc-uber-2.6.3.0-235.jar


## Criar a tabela no HIVE

In [7]:
import jaydebeapi

# Jar
hivejar = f"{base_path}/driver/{asset.name}"
driver = "org.apache.hive.jdbc.HiveDriver"
database = "db_olist"
# JDBC connection string
url=("jdbc:hive2://hive-server:10000/db_olist")
dadosbrutos_folder = '/datalake/dadosbrutos'

# Connect to HiveServer2 
conn = jaydebeapi.connect(jclassname=driver, url=url, jars=hivejar)
cursor = conn.cursor()

# Tabelas que serão criadas
tabelas = {'geolocation': f"""
            CREATE EXTERNAL TABLE {database}.geolocation
            (
                geolocation_zip_code_prefix INT COMMENT '5 primeiros digitos do CEP.',
                geolocation_lat DOUBLE COMMENT 'Latitude.',
                geolocation_lng DOUBLE COMMENT 'Longitude.',
                geolocation_city STRING COMMENT 'Cidade.',
                geolocation_state STRING COMMENT 'Estado.'
            ) COMMENT 'Tabela com CEPs brasileiros, e dados de geolocalização.'
            STORED AS ORC
            LOCATION '/datalake/dadosbrutos/olist_geolocation_dataset.orc/'
            """,
           
           'customers': f"""
            CREATE EXTERNAL TABLE {database}.customers
            (
                customer_id STRING COMMENT 'Chave para a tabela de ordens. Cada ordem tem um customer_id único.',
                customer_unique_id STRING COMMENT 'Identificador único do cliente.',
                customer_zip_code_prefix INT COMMENT 'Primeiros 5 digitos do CEP do cliente.',
                customer_city STRING COMMENT 'Cidade do endereço do cliente.',
                customer_state STRING COMMENT 'Estado do endereço do cliente.'
            ) COMMENT 'Tabela com clientes e seus dados de geolocalização.'
            STORED AS ORC
            LOCATION '/datalake/dadosbrutos/olist_customers_dataset.orc/'
            """,
           
           'order_items': f"""
            CREATE EXTERNAL TABLE {database}.order_items
            (
                order_id STRING COMMENT 'Identificador único da ordem.',
                order_item_id INT COMMENT 'Número sequencial identificando o número de itens incluídos na mesma ordem.',
                product_id STRING COMMENT 'Identificador único do produto.',
                seller_id STRING COMMENT 'Identificador único do vendedor.',
                shipping_limit_date TIMESTAMP COMMENT 'Mostra a data limite de entrega do vendedor para o parceiro logístico.',
                price DOUBLE COMMENT 'Preço do item.',
                freight_value DOUBLE COMMENT 'Valor do frete do item (se um pedido tiver mais de um item o valor do frete é dividido entre os itens).'
            ) COMMENT 'Tabela com dados dos itens comprados em uma ordem.'
            STORED AS ORC
            LOCATION '/datalake/dadosbrutos/olist_order_items_dataset.orc/'
            """,
           
           'order_payments': f"""
            CREATE EXTERNAL TABLE {database}.order_payments
            (
                order_id STRING COMMENT 'Identificador único da ordem.',
                payment_sequential INT COMMENT 'Um cliente pode pagar uma ordem com mais de um método de pagamento, a coluna indica a sequencia',
                payment_type STRING COMMENT 'Método de pagamento escolhido.',
                payment_installments INT COMMENT 'Número de parcelas escolhidas pelo cliente.',
                payment_value DOUBLE COMMENT 'Valor do pagamento.'
            ) COMMENT 'Tabela com dados de pagamento da ordem.'
            STORED AS ORC
            LOCATION '/datalake/dadosbrutos/olist_order_payments_dataset.orc/'
            """,
           
           'order_reviews': f"""
            CREATE EXTERNAL TABLE {database}.order_reviews
            (
                review_id STRING COMMENT 'Identificador único do review.',
                order_id STRING COMMENT 'Identificador único da ordem.',
                review_score INTEGER COMMENT 'Nota de 1 a 5 dada pelo cliente na pesquisa de satisfação',
                review_comment_title STRING COMMENT 'Título do comentário.',
                review_comment_message STRING COMMENT 'Mensagem do comentário.',
                review_creation_date TIMESTAMP COMMENT 'Data em que a pesquisa de satisfação foi envada.',
                review_answer_timestamp TIMESTAMP COMMENT 'Data em que a pesquisa de satisfação foi respondida.'
            ) COMMENT 'Tabela com dados dos reviews feitos pelos clientes.'
            STORED AS ORC
            LOCATION '/datalake/dadosbrutos/olist_order_reviews_dataset.orc/'
            """,
           
           'orders': f"""
            CREATE EXTERNAL TABLE {database}.orders
            (
                order_id STRING COMMENT 'Identificador único da ordem.',
                customer_id STRING COMMENT 'Identificador único do cliente',
                order_status STRING COMMENT 'Status da ordem (delivered, shipped, etc).',
                order_purchase_timestamp TIMESTAMP COMMENT 'Data da compra.',
                order_approved_at TIMESTAMP COMMENT 'Data de aprovação do pagamento.',
                order_delivered_carrier_date TIMESTAMP COMMENT 'Data em que o produto foi entregue ao parceiro logístico.',
                order_delivered_customer_date TIMESTAMP COMMENT 'Data em que o produto foi entregue ao cliente.',
                order_estimated_delivery_date TIMESTAMP COMMENT 'Data de entrega estimada informada ao cliente no momento da compra.'
            ) COMMENT 'Tabela com dados das ordens.'
            STORED AS ORC
            LOCATION '/datalake/dadosbrutos/olist_orders_dataset.orc/'
            """,
           
           'products': f"""
            CREATE EXTERNAL TABLE {database}.products
            (
                product_id STRING COMMENT 'Identificador único do produto.',
                product_category_name STRING COMMENT 'Nome da categoria do produto.',
                product_name_lenght DOUBLE COMMENT 'Número de caracteres extraídos do nome do produto.',
                product_description_lenght DOUBLE COMMENT 'Número de caracteres extraídos da descrição do produto.',
                product_photos_qty DOUBLE COMMENT 'Número de fotos publicadas do produto.',
                product_weight_g DOUBLE COMMENT 'Peso do produto medido em gramas.',
                product_length_cm DOUBLE COMMENT 'Comprimento do produto medido em centímetros.',
                product_height_cm DOUBLE COMMENT 'Altura do produto medida em centímetros.',
                product_width_cm DOUBLE COMMENT 'Largura do produto medida em centímetros.'
            ) COMMENT 'Tabela com dados dos produtos.'
            STORED AS ORC
            LOCATION '/datalake/dadosbrutos/olist_products_dataset.orc/'
            """,
           
           'sellers': f"""
            CREATE EXTERNAL TABLE {database}.sellers
            (
                seller_id STRING COMMENT 'Identificador único do vendedor.',
                seller_zip_code_prefix INT COMMENT '5 primeiros digitos do CEP do vendedor.',
                seller_city STRING COMMENT 'Cidade do endereço do vendedor.',
                seller_state STRING COMMENT 'Estado do endereço do vendedor.'
            ) COMMENT 'Tabela com dados dos vendedores.'
            STORED AS ORC
            LOCATION '/datalake/dadosbrutos/olist_sellers_dataset.orc/'
            """,
           
           'product_category_name_translation': f"""
            CREATE EXTERNAL TABLE {database}.product_category_name_translation
            (
                product_category_name STRING COMMENT 'Nome da categoria do produto em português .',
                product_category_name_english STRING COMMENT 'Nome da categoria do produto em inglês .'
            ) COMMENT 'Tabela de tradução do nome da categoria do produto.'
            STORED AS ORC
            LOCATION '/datalake/dadosbrutos/product_category_name_translation.orc/'
            """,
          }


for item in tabelas:
    # Dropar tabela caso ela já existir
    sql = f"""
        DROP TABLE IF EXISTS {database}.{item}
    """
    cursor.execute(sql)
    # Criar a tabela
    query = tabelas[item].strip('\n').strip('\t')
    sql = query
    cursor.execute(sql)
    